In [4]:
!pip install h2o

Defaulting to user installation because normal site-packages is not writeable
  Using cached h2o-3.46.0.7-py2.py3-none-any.whl.metadata (2.1 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
   ---------------------------------------- 0.0/265.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/265.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/265.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/265.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/265.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/265.9 MB ? eta -:--:--
   ---------------------------------------- 0.5/265.9 MB 349.5 kB/s eta 0:12:40
   ---------------------------------------- 0.5/265.9 MB 349.5 kB/s eta 0:12:40
   ---------------------------------------- 0.5/265.9 MB 349.5 kB/s eta 0:12:40
   ---------------------------------------- 0.5/265.9 MB 349.5 kB/s eta 0:12:40
   ---------------------------------------- 0

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\sajan\\AppData\\Local\\Temp\\pip-unpack-zmkw1emq\\h2o-3.46.0.7-py2.py3-none-any.whl'
Check the permissions.



In [5]:
import h2o
from h2o.automl import H2OAutoML
from sklearn.datasets import load_iris
import pandas as pd

ModuleNotFoundError: No module named 'h2o'

In [ ]:
# Load iris dataset
iris = load_iris(as_frame=True)
df = iris.frame
df['target'] = df['target'].astype(str)  # H2O requires target to be string for classification

# Start H2O cluster
h2o.init()

# Convert to H2O Frame
hf = h2o.H2OFrame(df)

# Features and target
x = hf.columns[:-1]
y = 'target'

# Train-test split
train, test = hf.split_frame(ratios=[0.8], seed=1)


In [ ]:
# Run AutoML
aml = H2OAutoML(max_models=10, seed=1)
aml.train(x=x, y=y, training_frame=train)


In [ ]:
# Predict on test set
pred = aml.leader.predict(test)

# Convert H2O frame to pandas DataFrame
pred_df = pred.as_data_frame()
actual_df = test[y].as_data_frame().reset_index(drop=True)

# Combine actual and predicted for dashboard
final_df = pd.concat([actual_df, pred_df], axis=1)
final_df.columns = ['Actual', 'Predicted_Class', 'Setosa_Prob', 'Versicolor_Prob', 'Virginica_Prob']

# Export to CSV (for use in Power BI or Tableau)
final_df.to_csv("iris_predictions_dashboard.csv", index=False)

print(final_df.head())
